In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from tokenizer import read_pgn_games
from chess_gpt import ChessGPT
from IPython.display import clear_output
from itertools import islice

import chess
import chess.engine
import chess.pgn
import time

In [4]:
_DB_PATH = "/Users/iudalov/code/ChessGPT/data/ficsgamesdb_202301_chess_nomovetimes_284330.pgn"
_ENGINE_PATH = "/Users/iudalov/code/ChessGPT/third-party/Stockfish/src/stockfish"

In [7]:

#all_games = [game for game in read_games(_DB_PATH)
all_games = list(islice(read_pgn_games(_DB_PATH), 100))

In [36]:
def animate_game(game, timeout_sec=3, show_eval=True, eval_time_limit=0.5):
    engine = None
    if show_eval:
        engine = chess.engine.SimpleEngine.popen_uci(_ENGINE_PATH)

    if isinstance(game, list):
        board = chess.Board()
        total_moves = len(game)
        moves = game
        title = "?"
    else:
        board = game.board()
        total_moves = len(list(game.mainline_moves()))
        moves= [str(move) for move in game.mainline_moves()]
        tille = "Result:", game.headers["Result"], f"{game.headers['White']} vs {game.headers['Black']}"

    for n, move in enumerate(moves):
        board.push_san(move)
        
        info = None
        if show_eval:
            info = engine.analyse(board, chess.engine.Limit(time=eval_time_limit))

        clear_output(wait=True)
        
        print(title)
        print(f"Move {n + 1}/{total_moves}: {move}")
        if show_eval:
            print("Eval:", info["score"], info["score"].wdl().white())

        display(board)
        time.sleep(timeout_sec)
    

    if engine is not None:
        engine.quit()

#game = "e2e4 e7e5 g1f3 b8c6 d2d4 e5d4 f3d4 d7d6 c1e3 c8f5 e4f5 g8f6 f1b5 f6d5 b1c3 f8e7 e1g1 e8g8 f1e1 f7f5 d1d2 f5f4 e3f4 f5f4 e1e2 g7g5 f4d6 f4g3 f2e3 c7d6 e3f4 d6f4 g3f4 g5f4 e3f4 a8f8 f4f5 d8d7 f5h3 c7c5 d4e6 d7e6 e3d5 e6d5 f5g6 f8f6 b3b4 c5d4 e2d4 e6f6 d1d4 f4g5 d4d5 g5e7 d5e7 e7f8 e7f7". split(" ")
#animate_game(all_games[21], show_eval=False)
#animate_game(game)

In [32]:
chess_gpt = ChessGPT("/Users/iudalov/code/ChessGPT/third-party/nanoGPT/out-chess/ckpt.pt")
stockfish = chess.engine.SimpleEngine.popen_uci(_ENGINE_PATH)

number of parameters: 0.80M


In [35]:
def show_board(board, timeout):
        clear_output(wait=True)
        display(board)
        time.sleep(timeout)

def stockfish_vs_chess_gpt(stockfish, chess_gpt, stockfish_is_white=True, num_samples=10, animate=False, animation_timeout=2):
    board = chess.Board()
    moves = list()
    stockfish_move = stockfish_is_white
    while True:
        if stockfish_move:            
            result = stockfish.play(board, chess.engine.Limit(time=0.1))
            board.push(result.move)
            moves.append(str(result.move))
        else:
            result = chess_gpt.next_moves(moves, n_moves=1, num_samples=num_samples)
            # print(f"### {' '.join(moves)}, options: {result}") # DEBUG
            if result is None:
                print(f"Game is too long {len(moves)}")
                return None, moves
            
            was_legal_move = False
            for item in result:
                if was_legal_move:
                    break
                try:
                    board.push_san(item[0])
                    moves.append(item[0])
                    was_legal_move = True
                except chess.IllegalMoveError as e:
                    #print(f"IllegalMoveError {item[0]}")
                    pass
                except chess.InvalidMoveError as e:
                    print(f"InvalidMoveError {item[0]}")

            if not was_legal_move:
                print("No legal moves was found", result)
                return None, moves
            

        if animate:
            show_board(board, animation_timeout)
            print(len(moves), " ".join(moves))
        
        stockfish_move = not stockfish_move

    if board.is_game_over():
        outcome = {"1-0": (1., 0.), "0-1": (0., 1.), "1/2-1/2": (0.5, 0.5)}[board.outcome().result()]
        print(f"Outcome {outcome}")
        return outcome, moves

n_tests = 10
good_games = 0
all_moves = list()
for i in range(n_tests):
    outcome, moves = stockfish_vs_chess_gpt(stockfish, chess_gpt)
    if outcome is not None:
        good_games += 1
    all_moves.append(moves)
    print(f"Game #{i}: {outcome}")

print(f"Summary {good_games}/{n_tests} were correct")
print("Avg game len:", sum([len(moves) for moves in  all_moves])/n_tests)


No legal moves was found [['c7b7'], ['c7b8'], ['f7g6'], ['g8h8'], ['h7g6'], ['c7b8'], ['f7g6'], ['g8f7'], ['h7g6'], ['d8c8']]
Game #0: None
InvalidMoveError <nop>
No legal moves was found [['g8h7'], ['g5h6'], ['g6h7'], ['g8h7'], ['g6e5'], ['g8h8'], ['g8h8'], ['a6a5'], ['<nop>'], ['g8f7']]
Game #1: None
No legal moves was found [['c7d6'], ['c8d7'], ['c7d6'], ['d8d6'], ['c8d7'], ['c6c5'], ['d8d6'], ['d7d6'], ['d8d6'], ['d8d6']]
Game #2: None
InvalidMoveError <nop>
No legal moves was found [['e7f7'], ['f8e8'], ['d8e8'], ['e7d8'], ['g7g6'], ['f6d5'], ['g7g6'], ['a6a5'], ['<nop>'], ['c8d6']]
Game #3: None
No legal moves was found [['b8b7'], ['b8d7'], ['b8d7'], ['a8b8'], ['b8d7'], ['b6b5'], ['a8c8'], ['a8d8'], ['d5d4'], ['a8c8']]
Game #4: None
No legal moves was found [['b8b7'], ['c7g3'], ['d7e8'], ['e7d8'], ['e7e3'], ['c7e8'], ['e8e6'], ['e8e5'], ['g8f8'], ['e6f5']]
Game #5: None
No legal moves was found [['c7d6'], ['c8d7'], ['c7d6'], ['d8d6'], ['c8d7'], ['c6c5'], ['d8d6'], ['d7d6'], ['d8d6